# Hyperparameter tuning

priority: 1 > 2 > 3 > 4

- 1 : $ \alpha $ : learning rate
- 2 : $ \beta \approx 0.9 $ : for momentum
- 4 : $ \beta_1 = 0.9 , \beta_2 = 0.999 , \epsilon = 10^{-8} $ : for Adam
- 3 : number of layers
- 2 : number of hidden units
- 3 : learning rate decay
- 2 : mini-batch size

使用随机分布的参数，不要用固定区间的 grid 模式。可以对每个 hyperparameter 获得更多的参考点。

获得第一批随机分布的参考点后，再对最优的区域，进行局部细化的随机测试。

## Scale for hyperparameters

随机的参数分布，有时不应该直接用原来数值进行随机分布。如

learning rate: $ \alpha $, 也许介于 0.0001 ~ 1 之间，需要不是这样的分布 

```
0.0001  0.1  0.2  0.3  0.4  0.5                       0.8  0.9  1.0
|      x|    |    | x  |     |x                x       |    | x  |  
```

而是取对数后的分布

```
0.0001          0.001           0.01            0.1              1
|      x         |         x     |x              |  x           x|
```

```
# using numpy
r = -4 * np.random.rand()
alpha = 10^r
```

$ 
a = \log_{10} 0.0001 = -4 \\
b = \log_{10} 1 = 0 \\
r \in [a, b] \\
\alpha = 10^r
$

for exponentially weighted averages

$
\beta \in [ 0.9, 0.999 ] \\
\big( 1 - \beta \big) \in [ 0.1, 0.0001 ]
$

## Batch Norm

在每一次 mini-batch 上，进行 $ Z^{[l ]}$ 的 Normalization，可以帮助每一层更独立地进行决策，较少收到别层的影响。  
也有 "一点" regularization 的效果。

这里会新增 parameter : $ \gamma, \beta $, (注意这个 beta 和 momentum, adam 的 beta 不同)

最后使用 $ \tilde{Z}^{[l]} $ 替代原来的 Z, 过程中 gamma, beta 内含了原来的 b 项，于是可以移除 b

也有些做法是对 $ A^{[l ]} $ 进行 Normalization.

注意在 test 的时候，没有 mini-batch，拿最后 training 获得的 mu, sigma (may use exponential weight average) 用在单个sample。

$$
\begin{cases}
\mu & = \frac{1}{m} \sum_i Z^{[l](i)} \\
\sigma^2 & = \frac{1}{m} \sum_i \big( Z^{[l]} - \mu \big)^2 \\
Z_{\text{norm}}^{[l]} & = \frac{Z^{[l]} - \mu}{\sqrt{\sigma^2 + \epsilon}} \\
\tilde{Z}^{[l]} & = \gamma Z_{\text{norm}}^{[l]} + \beta 
\end{cases}
$$